## pypdf

### PDF annotations extraction

In [1]:
from pypdf import PdfReader
import os


# Get the database root directory
try:
    root_dir = os.path.join(os.path.dirname(__file__), 'database', 'queplan_insurance')
except:
    print("This should not be in a Jupyter Notebook.")
    root_dir = os.path.join(os.path.dirname('.'), 'database', 'queplan_insurance')

# Get the list of PDF files in the directory
pdf_files = [os.path.join(root_dir, file) for file in os.listdir(root_dir) if file.endswith('.pdf')]


# PDF files annotations extraction
extractions_dict = {}
for pdf_file in pdf_files:
    reader = PdfReader(pdf_file)
    annotations_list = []

    for page in reader.pages:
        if "/Annots" in page:
            for annot in page["/Annots"]:
                obj = annot.get_object()
                annotation = {"subtype": obj["/Subtype"], "location": obj["/Rect"]}
                annotations_list.append(annotation)

    extractions_dict[os.path.basename(pdf_file)] = annotations_list


print(extractions_dict)


This should not be in a Jupyter Notebook.
{'POL120190177.pdf': [], 'POL320130223.pdf': [], 'POL320150503.pdf': [], 'POL320180100.pdf': [], 'POL320190074.pdf': [], 'POL320200071.pdf': [], 'POL320200214.pdf': [], 'POL320210063.pdf': [], 'POL320210210.pdf': []}


### Specialized Data Exploration

In [3]:
# PDF files data extraction
extractions_dict = {}
for pdf_file in pdf_files:
    reader = PdfReader(pdf_file)
    characteristics = {"characters": set(), "uppercase_lines": []}

    for page in reader.pages:
        text = page.extract_text()

        # Obtain the unique characters in the text
        distinct_chars = sorted(set(text))
        characteristics["characters"].update(distinct_chars)

        # Obtain the uppercase lines in the text
        uppercase_lines = [line.strip() for line in text.split("\n") if line.isupper()]
        characteristics["uppercase_lines"].extend(uppercase_lines)

    # Results storage
    extractions_dict[os.path.basename(pdf_file)] = characteristics

# Print the results
print(extractions_dict)


{'POL120190177.pdf': {'characters': {'6', 'g', 'M', 'R', 'I', '-', 'i', 'u', 'H', ';', 'Z', 'Í', '8', 'k', 'Ó', 'n', '.', '4', 'J', 't', 'f', 'j', 'Á', 'B', '9', '°', '"', '\n', 'e', 'O', 'F', 'P', 'V', 'y', 'ñ', '5', 'w', '/', '1', 'a', '2', 'm', '(', 'E', '3', 'o', 's', 'º', 'N', 'l', 'é', '0', 'U', 'D', 'b', 'S', 'X', 'C', 'É', 'c', 'G', 'z', 'L', 'ú', 'v', 'Q', 'r', 'ó', '7', 'q', 'Y', ':', 'x', ' ', ',', 'd', ')', 'á', 'h', 'T', 'í', 'p', 'A'}, 'uppercase_lines': ['PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS', 'ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO', 'ARTÍCULO 3°: LIMITACIONES DE LAS COBERTURAS', 'ARTÍCULO 6°: OBLIGACIONES DEL ASEGURADO', 'ARTÍCULO 7°: DECLARACIONES DEL CONTRATANTE Y DEL ASEGURADO', 'ARTÍCULO 10°: CALCULO DE LOS GASTOS REEMBOLSABLES', 'ARTÍCULO 11°: LIQUIDACIÓN DE LOS GASTOS, FORMA DE PAGO Y MONTO MÁXIMO DE GASTOS', 'REEMBOLSABLES', 'ARTÍCULO 12°: APLICACIÓN DEL DEDUCIBLE', 'ARTÍCULO 13°: VIGENCIA Y DURACIÓN DEL CONTRATO DE SEGURO', 'ARTÍCULO 1

In [8]:
extractions_dict.keys()
extractions_dict.keys()
extractions_dict['POL120190177.pdf'].keys()
extractions_dict['POL120190177.pdf']["characters"]
extractions_dict['POL120190177.pdf']["uppercase_lines"]

['PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS',
 'ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO',
 'ARTÍCULO 3°: LIMITACIONES DE LAS COBERTURAS',
 'ARTÍCULO 6°: OBLIGACIONES DEL ASEGURADO',
 'ARTÍCULO 7°: DECLARACIONES DEL CONTRATANTE Y DEL ASEGURADO',
 'ARTÍCULO 10°: CALCULO DE LOS GASTOS REEMBOLSABLES',
 'ARTÍCULO 11°: LIQUIDACIÓN DE LOS GASTOS, FORMA DE PAGO Y MONTO MÁXIMO DE GASTOS',
 'REEMBOLSABLES',
 'ARTÍCULO 12°: APLICACIÓN DEL DEDUCIBLE',
 'ARTÍCULO 13°: VIGENCIA Y DURACIÓN DEL CONTRATO DE SEGURO',
 'ARTÍCULO 14°: INCORPORACIÓN DE ASEGURADOS E INICIO DE LA VIGENCIA DE LA COBERTURA',
 'INDIVIDUAL',
 'ARTÍCULO 15°: TERMINACIÓN DEL CONTRATO',
 'ARTÍCULO 17°: MONEDA O UNIDAD DEL CONTRATO',
 'ARTICULO 18°: REHABILITACIÓN DE LA PÓLIZA',
 'ARTÍCULO 23°: DOMICILIO']

### Pattern Recognition and Total Pages

In [11]:
import os
# from PyPDF2 import PdfReader
import re


# Get the database root directory
try:
    root_dir = os.path.join(os.path.dirname(__file__), 'database', 'queplan_insurance')
except:
    print("This should not be in a Jupyter Notebook.")
    root_dir = os.path.join(os.path.dirname('.'), 'database', 'queplan_insurance')

# Get the list of PDF files in the directory
pdf_files = [os.path.join(root_dir, file) for file in os.listdir(root_dir) if file.endswith('.pdf')]

# Select the pattern to search
pattern = "cobertura"
pattern = "Artículo" # Case sensitive

# Function to count the pages of a PDF
def count_pages(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PdfReader(file)
        return len(reader.pages)

# Function to count the titles of a PDF
def count_pattern(pdf_file, pattern):
    with open(pdf_file, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return len(re.findall(r'\b' + pattern + r'\b', text))

# Process each PDF file
results = {}
for pdf_file in pdf_files:
    pdf_name = pdf_file.rsplit('\\')[-1]
    pages = count_pages(pdf_file)
    patterns = count_pattern(pdf_file, pattern)
    results[pdf_name] = {'pages': pages, f'<{pattern}>': patterns}

# Print results
for pdf, info in results.items():
    print(f"{pdf}: Pages: {info['pages']}, {pattern}: {info.get(f'<{pattern}>')}")


This should not be in a Jupyter Notebook.
POL120190177.pdf: Pages: 14, Artículo: 17
POL320130223.pdf: Pages: 47, Artículo: 2
POL320150503.pdf: Pages: 26, Artículo: 16
POL320180100.pdf: Pages: 25, Artículo: 15
POL320190074.pdf: Pages: 72, Artículo: 5
POL320200071.pdf: Pages: 25, Artículo: 17
POL320200214.pdf: Pages: 42, Artículo: 1
POL320210063.pdf: Pages: 6, Artículo: 20
POL320210210.pdf: Pages: 10, Artículo: 3


In [7]:
# root_dir = os.path.join(os.path.dirname(__file__), 'database', 'queplan_insurance')
# root_dir = os.path.join(os.path.dirname('.'), 'database', 'queplan_insurance')
# os.listdir(root_dir)

['POL120190177.pdf',
 'POL320130223.pdf',
 'POL320150503.pdf',
 'POL320180100.pdf',
 'POL320190074.pdf',
 'POL320200071.pdf',
 'POL320200214.pdf',
 'POL320210063.pdf',
 'POL320210210.pdf']